In [ ]:
# useful additional packages
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit.optimization.applications.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import aqua_globals
from qiskit.aqua import QuantumInstance
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

In [ ]:
# Generating a graph of 3 nodes
n = 3
num_qubits = n ** 2
ins = tsp.random_tsp(n, seed=123)
print('distance\n', ins.w)

# Draw the graph
G = nx.Graph()
G.add_nodes_from(np.arange(0, ins.dim, 1))
colors = ['r' for node in G.nodes()]

for i in range(0, ins.dim):
    for j in range(i+1, ins.dim):
        G.add_edge(i, j, weight=ins.w[i,j])

pos = {k: v for k, v in enumerate(ins.coord)}

draw_graph(G, colors, pos)

In [ ]:
#import the libraries
from qiskit import Aer, IBMQ
from qiskit.aqua.algorithms import QAOA
from qiskit.optimization.ising import max_cut, tsp
from qiskit.optimization.ising.common import sample_most_likely
import networkx as nx
import numpy as np

# Set up the 3-node graph
nodes = range(3)
distances = np.array([[0, 2, 1], [2, 0, 3], [1, 3, 0]])
graph = nx.Graph()
graph.add_nodes_from(nodes)
for i in range(len(distances)):
    for j in range(I):
        graph.add_edge(i, j, weight=distances[I, j])

# Setup the QAOA instance and the classical optimizer
qaoa = QAOA(max_cut.MaxCut(graph), p=2,
            optimizer=Aer.get_backend('statevector_simulator'))

# Run the algorithm and retrieve the results
results = qaoa.run(quantum_instance=Aer.get_backend('qasm_simulator'))

# Get the most likely solution
x = sample_most_likely(results['eigvecs'][0])
print('energy:', results['energy'])
print('solution:', x)

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
import numpy as np

# Create a classical and a quantum register
q = QuantumRegister(3)
c = ClassicalRegister(3)

# Create a quantum circuit
circuit = QuantumCircuit(q, c)

# Set up the circuit for QAOA
# Step 1 - Max-cut
# Initialize the angles
beta = np.random.uniform(0, 2*np.pi, 3)
gamma = np.random.uniform(0, 2*np.pi, 3)

# Add the first layer of gates
for i in range(3):
    circuit.rx(gamma[i], q[i])
    circuit.h(q[i])
    circuit.rz(beta[i], q[i])

# Add the second layer of gates
for i in range(3):
    circuit.cx(q[0], q[1])
    circuit.cx(q[1], q[2])

# Measure the circuit
circuit.measure(q, c)

# Execute the circuit
simulator = Aer.get_backend('qasm_simulator')
job = execute(circuit, simulator)
result = job.result()

# Print the results
print(result.get_counts(circuit))

{'101': 126, '100': 118, '001': 134, '110': 124, '000': 126, '010': 143, '111': 137, '011': 116}


In [ ]:
# import libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, execute
from qiskit.aqua.operators import WeightedPauliOperator

# define the circuit
q = QuantumRegister(3)
c = ClassicalRegister(3)
circuit = QuantumCircuit(q, c)

# define weights for the cost Hamiltonian
cost_weights = [1,1,1]

# define the cost Hamiltonian
cost_operator = WeightedPauliOperator(paulis=[
        (cost_weights[0], Pauli(label='X0 X1')),
        (cost_weights[1], Pauli(label='X1 X2')),
        (cost_weights[2], Pauli(label='X2 X0'))
    ])

# define the mixing Hamiltonian
mix_operator = WeightedPauliOperator(paulis=[
        (1, Pauli(label='Z0')),
        (1, Pauli(label='Z1')),
        (1, Pauli(label='Z2'))
    ])

# define the number of steps
steps = 3

# add the gates for the QAOA circuit
circuit.h(q)
for i in range(steps):
    circuit.barrier()
    circuit.rx(2*cost_operator.eval('111111'), q) 
    circuit.barrier()
    circuit.rz(2*mix_operator.eval('111111'), q)

# measure the result
circuit.measure(q,c)

# execute the circuit
backend = BasicAer.get_backend('qasm_simulator')
job = execute(circuit, backend, shots=1000)
result = job.result()

# print the result
print(result.get_counts(circuit))

In [3]:
# import libraries
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import Aer
from qiskit.aqua.algorithms import QAOA

# set up cost function
def cost_function(x):
    cost = 0
    N = 3 # Number of nodes
    weight_matrix = [[0, 10, 15],
                     [10, 0, 35],
                     [15, 35, 0]]
    for i in range(N):
        for j in range(N):
            if i != j:
                cost += weight_matrix[I][j] * x[i][j]
    return cost

# set up the quantum circuit
q = QuantumRegister(N**2)
c = ClassicalRegister(N**2)
qc = QuantumCircuit(q, c)

# set up QAOA
qaoa = QAOA(N, cost_function)

# add gates to the quantum circuit
for i in range(N):
    qc.h(q[i])
for i in range(N):
    qc.x(q[i])
for i in range(N):
    qc.rz(qaoa.angles[i], q[i])
for i in range(N):
    qc.h(q[i])
for i in range(N):
    qc.cx(q[i], q[i+1])
qc.measure(q, c)

# execute the circuit
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1000)
result = job.result()

# get the final state
final_state = result.get_counts()

# print the final state and the cost
print("final state:", final_state)
print("cost:", cost_function(final_state))

ModuleNotFoundError: No module named 'qiskit.algorithms.minimum_eigensolvers.QAOA'

In [4]:
import qiskit
qiskit.__version__

'0.22.1'